In [1]:
import pandas as pd
from collections import Counter
import nltk
import re
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

keywordsJson = pd.read_json('../data/depression_synonyms.json', orient='records')
keywords = list(keywordsJson['depression'])
keywordsdict = {keyword: 0 for keyword in keywords}
print(keywords)

['affect lack', 'affect unhappy', 'affects lack', 'anhedonia', 'anhedonias', 'apathetic', 'apathetic behavior', 'apathetic behaviour', 'apathy', 'cannot see a future', 'decreased interest', 'decreased mood', 'demoralisation', 'demoralization', 'depressed', 'depressed mood', 'depressed state', 'depressing', 'depression', 'depressions', 'depression emotional', 'depression mental', 'depression mental function', 'depression moods', 'depression psychic', 'depression symptom', 'depressive state', 'depressive symptom', 'depressive symptoms', 'despair', 'diminished pleasure', 'emotional depression', 'emotional indifference', 'emotionally apathetic', 'emotionally cold', 'emotionally detached', 'emotionally distant', 'emotionally subdued', 'feeling blue', 'feeling despair', 'feeling depressed', 'feeling down', 'feeling empty', 'feeling helpless', 'feeling hopeless', 'feeling isolated', 'feeling lonely', 'feeling of loss of feeling', 'feeling lost', 'feeling low', 'feeling of despair', 'feeling o

[nltk_data] Downloading package wordnet to /Users/bram/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
df = pd.read_json('../data/splitData/postdataLinesSplit.json',lines=True)

def preprocess(post):
    preprocessedWords = []
    lemmatizer = WordNetLemmatizer()
    post = post[0]
    post = post.lower()
    post = re.sub(r'[^\w\s]', '', post)
    words = post.split()
    for word in words:
        lemmatized = lemmatizer.lemmatize(word)
        preprocessedWords.append(lemmatized)
    
    return preprocessedWords

prediction = {'prediction':[]}
vector_dicts = []

for index, row in df.iterrows():
    DepressionWordCount = 0
    text = row['text']
    preProcessed = preprocess(text)
    wordCounts = dict(Counter(preProcessed))
    keywordCounts = keywordsdict
    keywordCounts['id'] = index
    for word in keywords:
        try:
            keywordCounts[word] = wordCounts[word]
        except: 
            continue
    vector_dicts.append(keywordCounts)


vector_df = pd.DataFrame(vector_dicts)
display(vector_df)
df = pd.concat([df, vector_df], axis=1)
df.to_json("../data/vectorData/DictionaryBasedVectors.json", orient='records', lines=True)

,affect lack,affect unhappy,affects lack,anhedonia,anhedonias,apathetic,apathetic behavior,apathetic behaviour,apathy,cannot see a future,...,worsening depression obessionscompulsions,worsening depressive,worsening depressive symptoms,worsening refractory depression,worthless hopeless,worthlessness guilt,worthlessness hopelessness,worthlessness low energy,worthlessnessguilt,id
0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,796
1,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,796
2,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,796
3,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,796
4,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,796
793,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,796
794,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,796
795,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,796


In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import numpy as np

In [4]:
df = pd.read_json('../data/vectorData/DictionaryBasedVectors.json', orient='records', lines=True)

(df['gender'] == 'm') & (df['split'] == 'train')

X_train = df.loc[(df['split'] == 'train'), [str(i) for i in keywords]]
X_test = df.loc[(df['split'] == 'test'), [str(i) for i in keywords]]
# X_testm = df.loc[(df['gender'] == 'm') & (df['split'] == 'test'), [str(i) for i in range(712)]]
# X_testf = df.loc[(df['gender'] == 'f') & (df['split'] == 'test'), [str(i) for i in range(712)]]

y_train = df.loc[(df['split'] == 'train'), ['label']]
y_test = df.loc[(df['split'] == 'test'), ['label']]
# y_testm = df.loc[(df['gender'] == 'm') & (df['split'] == 'test'), ['label']]
# y_testf = df.loc[(df['gender'] == 'f') & (df['split'] == 'test'), ['label']]

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
# X_testm = scaler.transform(X_testm)
# X_testf = scaler.transform(X_testf)

y_train = y_train.values.ravel()

param_grid = {'C': np.logspace(-3, 3, 7),  
              'gamma': np.logspace(-3, 3, 7), 
              'kernel': ['rbf']}  

cv = 5
scoring = 'accuracy'

grid = GridSearchCV(SVC(random_state=99, probability=True, class_weight='balanced'), param_grid, scoring=scoring, cv=cv, n_jobs=-1)
grid.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=SVC(class_weight='balanced', probability=True,
                           random_state=99),
             n_jobs=-1,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'kernel': ['rbf']},
             scoring='accuracy')

In [20]:
y_pred = grid.predict(X_test)

testSet = df.loc[(df['split'] == 'test')]

testSet['prediction'] = y_pred


testSet.to_json('../data/predictionData/DictionaryBasedPred.json',orient='records',lines=True)

['depression']


/var/folders/qw/2ytfcz3d78qfsgsvq73r4dx80000gn/T/ipykernel_46818/2432990572.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testSet['prediction'] = y_pred
